<a href="https://colab.research.google.com/github/Bhairavsingh/ASL_Alphabets_Detection/blob/master/ASL_Alphabets_Classification_Rev_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#Loading required libraries.
import pandas as pd
import numpy as np
import re
import string
from pprint import pprint
from collections import Counter
import keras
import os
import json
import sklearn
import csv

import tensorflow as tf
#!pip install tensorflow-gpu
#import tensorflow-gpu as tfg

from keras.datasets import imdb
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense, Dropout
from keras import layers
from keras.layers import LSTM
from keras.layers import LeakyReLU, UpSampling1D, Input, Reshape, Activation, Lambda, AveragePooling1D,AveragePooling2D
from keras.layers import Convolution2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import RMSprop

import matplotlib.pyplot as plt
from matplotlib import image
#!pip install pypng
#import png
#!pip install Pillow
import PIL
from PIL import Image
from glob import glob

from numpy import save
from numpy import load
import time


In [ ]:


#Function for reading image paths and creating labels of images based on the folder names it is stored in.
def image_listing(directory):
  #Listing folder names from given directory.
  directory = directory + '/*/'
  Directory = glob(directory)
  File_names = []
  #Creating list of list of images stored in each folder of directory.
  for i in range(len(Directory)):
    Folders = Directory[i] + '*'
    File_names.append(glob(Folders))

  #Listing all file path in one list and creating a list of labels.
  file_path = []
  labels = []
  for i in range(len(File_names)):
    for j in range(len(File_names[i])):
      #Reading each file path.
      x = File_names[i][j]
      file_path.append(x)
      #Reading folder name where image is stored for labeling it.
      if x[62:65] == 'del':
        labels.append(x[62:65])
      elif x[62:67] == 'space':
        labels.append(x[62:67])
      elif x[62:69] == 'nothing':
        labels.append(x[62:69])
      else:
        labels.append(x[62])
  return file_path, labels



#Function for creating train, validation and testsets according to the given percentage values.
def set_creator(directory, trainset_percentage = 0.8, validationset_percentage = 0.1, testset_percentage = 0.1):
  #Reading path and labels of all images.
  file_names, labels = image_listing(directory)

  #Image names meta data.
  Data_A = file_names[15008 : 18007]
  Data_B = file_names[21008 : 24015]
  Data_C = file_names[27016 : 30023]
  Data_D = file_names[0 : 2999]
  Data_E = file_names[12008 : 15007]
  Data_F = file_names[18008 : 21007]
  Data_G = file_names[6000 : 9007]
  Data_H = file_names[24016 : 27015]
  Data_I = file_names[9008 : 12007]
  Data_J = file_names[33024 : 36023]
  Data_K = file_names[51024 : 54031]
  Data_L = file_names[45024 : 48023]
  Data_M = file_names[42024 : 45023]
  Data_N = file_names[57032 : 60031]
  Data_O = file_names[48024 : 51023]
  Data_P = file_names[54032 : 57031]
  Data_Q = file_names[30024 : 33023]
  Data_R = file_names[36024 : 39023]
  Data_S = file_names[63032 : 66031]
  Data_T = file_names[81032 : 84031]
  Data_U = file_names[66032 : 69031]
  Data_V = file_names[72032 : 75031]
  Data_W = file_names[84032 :]
  Data_X = file_names[78032 : 81031]
  Data_Y = file_names[60032 : 63031]
  Data_Z = file_names[75032 : 78031]
  Data_nothing = file_names[39024 : 42023]
  Data_del = file_names[3000 : 5999]
  Data_space = file_names[69032 : 72031]

  #Labels metadata.
  Labels_A = labels[15008 : 18007]
  Labels_B = labels[21008 : 24015]
  Labels_C = labels[27016 : 30023]
  Labels_D = labels[0 : 2999]
  Labels_E = labels[12008 : 15007]
  Labels_F = labels[18008 : 21007]
  Labels_G = labels[6000 : 9007]
  Labels_H = labels[24016 : 27015]
  Labels_I = labels[9008 : 12007]
  Labels_J = labels[33024 : 36023]
  Labels_K = labels[51024 : 54031]
  Labels_L = labels[45024 : 48023]
  Labels_M = labels[42024 : 45023]
  Labels_N = labels[57032 : 60031]
  Labels_O = labels[48024 : 51023]
  Labels_P = labels[54032 : 57031]
  Labels_Q = labels[30024 : 33023]
  Labels_R = labels[36024 : 39023]
  Labels_S = labels[63032 : 66031]
  Labels_T = labels[81032 : 84031]
  Labels_U = labels[66032 : 69031]
  Labels_V = labels[72032 : 75031]
  Labels_W = labels[84032 :]
  Labels_X = labels[78032 : 81031]
  Labels_Y = labels[60032 : 63031]
  Labels_Z = labels[75032 : 78031]
  Labels_nothing = labels[39024 : 42023]
  Labels_del = labels[3000 : 5999]
  Labels_space = labels[69032 : 72031]

  Data_file_meta_info = [Data_A, Data_B, Data_C, Data_D, Data_E, Data_F, Data_G, Data_H, Data_I, Data_J, Data_K, Data_L, Data_M, Data_N, Data_O, 
                        Data_P, Data_Q, Data_R, Data_S, Data_T, Data_U, Data_V, Data_W, Data_X, Data_Y, Data_Z, Data_nothing, Data_del, Data_space]

  Label_file_meta_info = [Labels_A, Labels_B, Labels_C, Labels_D, Labels_E, Labels_F, Labels_G, Labels_H, Labels_I, Labels_J, Labels_K, Labels_L, Labels_M, Labels_N, Labels_O, 
                          Labels_P, Labels_Q, Labels_R, Labels_S, Labels_T, Labels_U, Labels_V, Labels_W, Labels_X, Labels_Y, Labels_Z, Labels_nothing, Labels_del, Labels_space]

  #Declaring lists for all sets.
  train_file_names = []
  train_labels = []
  val_file_names = []
  val_labels = []
  test_file_names = []
  test_labels = []
  
  #Loop for equally extracting images and respective labels from each class.
  for i in range(len(Data_file_meta_info)):
    #Training value and validation value.
    train_value = int(len(Data_file_meta_info[i]) * trainset_percentage)
    val_value = train_value + int(len(Data_file_meta_info[i]) * validationset_percentage)
    #Train sets.
    train_file_names.extend(Data_file_meta_info[i][ : train_value])
    train_labels.extend(Label_file_meta_info[i][ : train_value])
    #Validation sets.
    val_file_names.extend(Data_file_meta_info[i][train_value : val_value])
    val_labels.extend(Label_file_meta_info[i][train_value : val_value])
    #Test sets.
    test_file_names.extend(Data_file_meta_info[i][val_value : ])
    test_labels.extend(Label_file_meta_info[i][val_value : ])
  return train_file_names, train_labels, val_file_names, val_labels, test_file_names, test_labels



#Function for reading the images.
def image_reader(image_path, normalizing_value = 255):
  #Reading image.
  data_raw = image.imread(image_path)
  #print(data_raw.dtype)
  #print(data_raw.shape)
  plt.imshow(data_raw)
  plt.show()
  #Normalizing the image array using max pixel value.
  normalized_data = np.array(data_raw/normalizing_value)
  return normalized_data



#Function for reading list of paths and reading images to get arrays.
def read_image_get_array(file_paths, normalizing_value = 255):
  lol = []
  for i in range(len(file_paths)):
    x = image_reader(file_paths[i], normalizing_value)
    lol.append(x)
  arrays = np.array(lol)
  return arrays



'''
#Reading all file path names and respective labels.
file_names, labels = image_listing(directory = '/content/drive/My Drive/Colab_Datasets/ASL/asl_alphabet_train')

#Writing the lyrics in a csv file!
#file = open('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Image_paths.csv','w+', encoding = 'utf-8', newline = '\n')
file = open('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Image_labels.csv','w+', encoding = 'utf-8', newline = '\n')
with file:
    objects = csv.writer(file)
    objects.writerows(labels)
file.close()
'''



#Index dictonary for classes and its key integer values.
meta_dict_classes = {'nothing' : 0, 'A' : 1, 'B' : 2, 'C' : 3, 'D' : 4, 'E' : 5, 'F' : 6, 'G' : 7, 'H' : 8, 'I' : 9, 'J' : 10, 'K' : 11, 'L' : 12, 'M' : 13, 
                       'N' : 14, 'O' : 15, 'P' : 16, 'Q' : 17, 'R' : 18, 'S' : 19, 'T' : 20, 'U' : 21, 'V' : 22, 'W' : 23, 'X' : 24, 'Y' : 25, 'Z' : 26, 'del' : 27, 'space' : 28}
#meta_pred_dict_classes = {value:key for key, value in meta_dict_classes.items()}
meta_pred_dict_classes = {0: 'nothing', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M', 
                          14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z', 27: 'del', 28: 'space'}


#Function for converting alphabets to key integers and converting to categorical array (one hot encodding).
def converting_to_intkeys(label_list, meta_dict_classes = meta_dict_classes):
  for n, i in enumerate(label_list):
    label_list[n] = meta_dict_classes[i]
  label_array = tf.keras.utils.to_categorical(label_list)
  return label_array


#Function for predicting given image using the trained model.
def predict_image(test_path, model, meta_pred_dict_classes = meta_pred_dict_classes):
  #Reading image and generating normalized numpy array.
  test_image = read_image_get_array(file_paths = [test_path], normalizing_value = 255)
  #Displaying image.
  plt.imshow(image.imread(test_path))
  plt.show()
  #Predicting class.
  result = model.predict(test_image)
  #Class integer key.
  predicted_class_key = np.argmax(result)
  #print(predicted_class_key)
  #Finding classes based on predefined dictonary.
  predicted_class = meta_pred_dict_classes[predicted_class_key]
  return predicted_class


#Function for predicting class of given image using class name and image number from database using trained model.
def customized_predict(class_name, image_number, model):
  test_path = '/content/drive/My Drive/Colab_Datasets/ASL/asl_alphabet_train/' + class_name + '/' + class_name + image_number + '.jpg'
  predicted_class = predict_image(test_path = test_path, model = model)
  print("Predicted class of above image is: ", predicted_class)
  return predicted_class




In [ ]:


#Loading training sets.
training_array_1 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_1.npy', allow_pickle = True)
training_array_2 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_2.npy', allow_pickle = True)
training_array_3 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_3.npy', allow_pickle = True)
training_array_4 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_4.npy', allow_pickle = True)
training_array_5 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_5.npy', allow_pickle = True)
#training_array_6 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_6.npy', allow_pickle = True)
#training_array_7 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_7.npy', allow_pickle = True)
#training_array_8 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_8.npy', allow_pickle = True)
#training_array_9 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_9.npy', allow_pickle = True)
training_array_10 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_10.npy', allow_pickle = True)

training_array_11 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_11.npy', allow_pickle = True)
training_array_12 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_12.npy', allow_pickle = True)
training_array_13 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_13.npy', allow_pickle = True)
training_array_14 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_14.npy', allow_pickle = True)
training_array_15 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_15.npy', allow_pickle = True)
training_array_16 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_16.npy', allow_pickle = True)
training_array_17 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_17.npy', allow_pickle = True)
training_array_18 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_18.npy', allow_pickle = True)
training_array_19 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_19.npy', allow_pickle = True)
training_array_20 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_20.npy', allow_pickle = True)

training_array_21 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_21.npy', allow_pickle = True)
training_array_22 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_22.npy', allow_pickle = True)
training_array_23 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_23.npy', allow_pickle = True)
training_array_24 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_24.npy', allow_pickle = True)
training_array_25 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_25.npy', allow_pickle = True)
training_array_26 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_26.npy', allow_pickle = True)
training_array_27 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_27.npy', allow_pickle = True)
training_array_28 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_28.npy', allow_pickle = True)
training_array_29 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_29.npy', allow_pickle = True)

'''
#Concatenating all training arrays in one array.
training_array = np.concatenate((training_array_1, training_array_2, training_array_3, training_array_4, training_array_5, 
                                 training_array_6, training_array_7, training_array_8, training_array_9, training_array_10,
                                 training_array_11, training_array_12, training_array_13, training_array_14, training_array_15, 
                                 training_array_16, training_array_17, training_array_18, training_array_19, training_array_20,
                                 training_array_21, training_array_22, training_array_23, training_array_24, training_array_25, 
                                 training_array_26, training_array_27, training_array_28, training_array_29), axis = 0)
print("Shape of training set: ", training_array.shape)
'''


'\n#Concatenating all training arrays in one array.\ntraining_array = np.concatenate((training_array_1, training_array_2, training_array_3, training_array_4, training_array_5, \n                                 training_array_6, training_array_7, training_array_8, training_array_9, training_array_10,\n                                 training_array_11, training_array_12, training_array_13, training_array_14, training_array_15, \n                                 training_array_16, training_array_17, training_array_18, training_array_19, training_array_20,\n                                 training_array_21, training_array_22, training_array_23, training_array_24, training_array_25, \n                                 training_array_26, training_array_27, training_array_28, training_array_29), axis = 0)\nprint("Shape of training set: ", training_array.shape)\n'

In [ ]:

#Loading validation sets.
validation_array = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/validation_array.npy', allow_pickle = True)

#Loading label sets.
y_train = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/y_train.npy', allow_pickle = True)
y_val = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/y_val.npy', allow_pickle = True)





In [ ]:
training_array_11.shape

(500, 200, 200, 3)

In [ ]:

training_array_19 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_19.npy', allow_pickle = True)


In [ ]:

#training_array_7 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_7.npy', allow_pickle = True)
#training_array_8 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_8.npy', allow_pickle = True)
training_array_9 = np.load('/content/drive/My Drive/Colab_Datasets/ASL/ASL_Alphabet_Numpy_Data/training_array_9.npy', allow_pickle = True)


ValueError: ignored